In [8]:
import re 
import json
import requests
from bs4 import BeautifulSoup
from datetime import datetime 
import os
import pandas as pd
import shutil
import yaml
import pickle
#from helium import *
import time
import traceback
from agenda_tools import get_schedule, download_images, get_category
from requests_html import HTMLSession
from IPython.display import Image
import urllib

session = HTMLSession()

from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium import webdriver
from agenda_tools.get_schedule import remove_blank_spaces

from requests.exceptions import ConnectionError

In [3]:
first_page_pattern = re.compile(r'.*?pg=1')

pattern_city = re.compile('https:.*/schools/(.*)/.*')

# Proxy

In [182]:
with open('/home/cesar/Downloads/good_proxies.txt','r') as f:
    lines = f.readlines()

In [89]:
with open('/home/cesar/Downloads/socks4_proxies.txt','r') as f:
    lines2 = f.readlines()

In [90]:
lines = lines+lines2

In [91]:
len(lines)

1045

# Process

In [110]:
[ i for i in list(set(list_cities)) if i != None][97]

'https://www.edarabia.com/schools/jakarta/'

In [140]:
len(set(list_cities))

258

In [174]:
133+84

217

In [173]:
for idx, i in enumerate([ i for i in list_cities[133:] if i != None]):
    try:
        if re.match(pattern_city,i).group(1) == 'swansea':
            print(idx)
    except:
        pass

84


In [5]:
link = 'https://www.edarabia.com/schools/'

In [9]:
request_main_link = requests.get(link)
main_soup = BeautifulSoup(request_main_link.text,'lxml')
list_cities = [i.a.get('href') for li in main_soup.find_all('ul',attrs={'class':"fourth-col"}) for i in li]

In [10]:
list_cities

['https://www.edarabia.com/schools/abu-dhabi/',
 'https://www.edarabia.com/schools/ajman/',
 'https://www.edarabia.com/schools/al-ain/',
 'https://www.edarabia.com/schools/dubai/',
 'https://www.edarabia.com/schools/fujairah/',
 'https://www.edarabia.com/schools/rak/',
 'https://www.edarabia.com/schools/sharjah/',
 'https://www.edarabia.com/schools/adelaide/',
 'https://www.edarabia.com/schools/brisbane/',
 'https://www.edarabia.com/schools/canberra/',
 'https://www.edarabia.com/schools/darwin/',
 'https://www.edarabia.com/schools/hobart/',
 'https://www.edarabia.com/schools/melbourne/',
 'https://www.edarabia.com/schools/new-south-wales/',
 'https://www.edarabia.com/schools/perth/',
 'https://www.edarabia.com/schools/queensland/',
 'https://www.edarabia.com/schools/sydney/',
 'https://www.edarabia.com/schools/tasmania/',
 'https://www.edarabia.com/schools/victoria/',
 'https://www.edarabia.com/schools/vienna/',
 'https://www.edarabia.com/schools/adliya/',
 'https://www.edarabia.com/sc

In [178]:
def get_page(last_page,idx_p, city):
    data = []
    if last_page == None:
        for second_proxy in lines:
            link = '{}'.format(city)
            try:
                rq = requests.get(link,proxies={'http':second_proxy,'https':second_proxy},headers=header,timeout=3)
                #rq = requests.get(link)
                soup_s = BeautifulSoup(rq.text,'lxml')
                print('\t',rq.status_code)
                if soup_s.find_all('div',attrs={'class':"list-items"}):
                    list_links = [i.find('div',attrs={'class':"row"}).find('div',attrs={'class':"col-md-10 col-sm-10 col-xs-10"}) for i in soup.find_all('div',attrs={'class':"list-items"}) if i is not None]
                    links_page = [i.h5.a.get('href') for i in list_links if i is not None]
                    data.append(links_page)
                    print('Success')
                    break
            except ConnectionError as cx:
                pass
            except Exception as ex:
                print('\tFailed second round')
                print(ex)
        return data

    for i in range(1,last_page+1):
        print('\t Page Index:',i)
        for second_proxy in lines:
            link = '{}?pg={}'.format(city,i)
            if re.match(first_page_pattern,link):
                link = '{}'.format(city)
           # print('\t Link Page:',link)
            try:
                rq = requests.get(link,proxies={'http':second_proxy,'https':second_proxy},headers=header,timeout=3)
                #rq = requests.get(link)
                soup_s = BeautifulSoup(rq.text,'lxml')
                print('\t',rq.status_code)
                if soup_s.find_all('div',attrs={'class':"list-items"}):
                    list_links = [i.find('div',attrs={'class':"row"}).find('div',attrs={'class':"col-md-10 col-sm-10 col-xs-10"}) for i in soup.find_all('div',attrs={'class':"list-items"}) if i is not None]
                    links_page = [i.h5.a.get('href') for i in list_links if i is not None]
                    data.append(links_page)
                    print('Success')
                    break
            except ConnectionError as cx:
                pass
            except Exception as ex:
                print('\tFailed second round')
                print(ex)
    return data

In [83]:
len(lines)

57

In [ ]:
header = {'user-agent':'Mozilla/5.0 (Windows Phone 10.0; Android 4.2.1; Microsoft; RM-1127_16056) AppleWebKit/537.36(KHTML, like Gecko) Chrome/42.0.2311.135 Mobile Safari/537.36 Edge/12.10536'}

In [165]:
data_faltante = []

In [183]:
# Extraer link con ciudades
#data_links = []
proxy_num = 0
for city in list_cities[217:]:
    print(city)
    if proxy_num +1 == len(lines):
        print('Se acabaron los proxies')
    for idx_p , proxy in enumerate(lines[proxy_num:]):
        #idx_p += proxy_num
        try:
            r = requests.get(city,proxies={'http':proxy,'https':proxy},headers=header,timeout=3)
            soup = BeautifulSoup(r.text,'lxml')
            print(r.status_code)
            print(soup.find('a',attrs={'title':"Last"}))
            if soup.find('h5'):
                try:
                    last_page = int(soup.find('a',attrs={'title':"Last"}).get('data-page'))
                except:
                    last_page = None
                print(last_page)
                data = get_page(last_page,idx_p, city)
                data_dict = {'City':re.match(pattern_city,city).group(1),'Data':data}
                data_faltante.append(data_dict)
                break
            else:
                raise Exception('The page dont load')
                
        except ConnectionError as cx:
           # print("Failed Conecction")
            pass
        except Exception as ex:
            print('Failed')
            print(ex)
        proxy_num = idx_p
        #time.sleep(5)

https://www.edarabia.com/schools/swansea/
200
None
Failed
The page dont load
200
None
None
	Failed second round
HTTPSConnectionPool(host='www.edarabia.com', port=443): Read timed out. (read timeout=3)
	 200
https://www.edarabia.com/schools/american/
200
<a data-page="49" href="#" title="Last">Last</a>
49
	 Page Index: 1
	 Page Index: 2
	Failed second round
HTTPSConnectionPool(host='www.edarabia.com', port=443): Read timed out. (read timeout=3)
	 Page Index: 3
	 200
Success
	 Page Index: 4
	Failed second round
HTTPSConnectionPool(host='www.edarabia.com', port=443): Read timed out. (read timeout=3)
	Failed second round
HTTPSConnectionPool(host='www.edarabia.com', port=443): Read timed out. (read timeout=3)
	 Page Index: 5
	Failed second round
HTTPSConnectionPool(host='www.edarabia.com', port=443): Read timed out. (read timeout=3)
	 200
	Failed second round
HTTPSConnectionPool(host='www.edarabia.com', port=443): Read timed out. (read timeout=3)
	 Page Index: 6
	 200
	 Page Index: 7
	Faile

In [175]:
for i in list_cities[217:]:
    print(i)

https://www.edarabia.com/schools/swansea/
https://www.edarabia.com/schools/american/
https://www.edarabia.com/schools/arabic/
https://www.edarabia.com/schools/australian/
https://www.edarabia.com/schools/boarding/
https://www.edarabia.com/schools/british/
https://www.edarabia.com/schools/canadian/
https://www.edarabia.com/schools/cbse/
https://www.edarabia.com/schools/cfe/
https://www.edarabia.com/schools/cisce/
https://www.edarabia.com/schools/egyptian/
https://www.edarabia.com/schools/filipino/
https://www.edarabia.com/schools/french/
https://www.edarabia.com/schools/gcse/
https://www.edarabia.com/schools/german/
https://www.edarabia.com/schools/international-baccalaureate/
https://www.edarabia.com/schools/igcse/
https://www.edarabia.com/schools/indian/
https://www.edarabia.com/schools/iranian/
https://www.edarabia.com/schools/islamic/
https://www.edarabia.com/schools/italian/
https://www.edarabia.com/schools/japanese/
https://www.edarabia.com/schools/moe/
https://www.edarabia.com/sc

In [184]:
data_faltante

[{'City': 'izmir',
  'Data': [['https://www.edarabia.com/9988/ozel-izmir-amerikan-koleji/',
    'https://www.edarabia.com/9100/ekin-koleji/',
    'https://www.edarabia.com/9145/isikkent-egitim-kampusu/',
    'https://www.edarabia.com/9185/izmir-buyuk-cigli-ozel-turk-anadolu-lisesi/',
    'https://www.edarabia.com/9196/izmir-ozel-turk-koleji/',
    'https://www.edarabia.com/10303/ozel-gelisim-ilkogretim-okulu/',
    'https://www.edarabia.com/9989/ozel-italyan-ana-ve-ilkokulu/',
    'https://www.edarabia.com/10002/ozel-izmir-sev-ilkogretim-okulu/']]},
 {'City': 'malatya',
  'Data': [['https://www.edarabia.com/ted-malatya-college-malatya-turkey/']]},
 {'City': 'mersin',
  'Data': [['https://www.edarabia.com/9054/cag-koleji/',
    'https://www.edarabia.com/girne-american-university-mersin-turkey/',
    'https://www.edarabia.com/9305/odtu-gelistirme-vakfi-ozel-mersin-ilkogretim-okulu/',
    'https://www.edarabia.com/9956/ozel-cag-lisesi/',
    'https://www.edarabia.com/10513/ozel-tarsus-ame

In [15]:
lines = 'a'

In [23]:
data_links

[{'City': 'abu-dhabi', 'Data': []},
 {'City': 'ajman', 'Data': []},
 {'City': 'al-ain', 'Data': []},
 {'City': 'dubai', 'Data': []}]

In [35]:
soup.find('a',attrs={'title':"Last"}) != None

True

In [36]:
int(soup.find('a',attrs={'title':"Last"}).get('data-page'))

9

In [57]:
def aste(a):
    assert a == '3', 'Error'

In [72]:
list_cities[41]

'https://www.edarabia.com/schools/manchester/'

In [125]:
# # Extraer link con ciudades
# # data_links = []
# data_failed = []
# for city in list_cities[41:]:
#     print(city)
#     for idx_p, proxy in enumerate(lines):
#         try:
#             r = requests.get(city)
#             soup = BeautifulSoup(r.text,'lxml')
#             print(r.status_code)
# #             print(soup.find('a',attrs={'title':"Last"}))
#             print(soup.find('a',attrs={'title':"Last"}) != None)
#             if soup.find('h5') != None:
#                 last_page = int(soup.find('a',attrs={'title':"Last"}).get('data-page'))
#                 print(last_page)
#                 data = get_page(last_page,idx_p, city)
#                 data_dict = {'City':re.match(pattern_city,city).group(1),'Data':data}
#                 data_links.append(data_dict)
#                 break
#             else:
#                 assert soup.find('a',attrs={'title':"Last"}) != None, 'Error en conexion'
                
#         except AssertionError as ax:
#             print(ax)
#             data_failed.append(city)
                
#         except ConnectionError as cx:
#             print("Failed Conecction")
#             pass
#         except Exception as ex:
#             print('Failed')
#             print(ex)
#         time.sleep(5)

https://www.edarabia.com/schools/manchester/
200
False
Error en conexion
200
False
Error en conexion
200
False
Error en conexion
200
False
Error en conexion
200
False
Error en conexion
200
False
Error en conexion
200
False
Error en conexion
200
False
Error en conexion
200
False
Error en conexion
200
False
Error en conexion
200
False
Error en conexion
200
False
Error en conexion
200
False
Error en conexion
200
False
Error en conexion
200
False
Error en conexion


KeyboardInterrupt: 

In [63]:
data_links

[{'City': 'abu-dhabi',
  'Data': [['https://www.edarabia.com/6186/horizon-private-school-khalifa-city/',
    'https://www.edarabia.com/137352/virginia-international-private-school-abu-dhabi-uae/',
    'https://www.edarabia.com/259/al-dhafra-private-school/',
    'https://www.edarabia.com/137366/global-indian-international-school-abu-dhabi-uae/',
    'https://www.edarabia.com/6388/british-international-school-abu-dhabi-uae/',
    'https://www.edarabia.com/82124/cranleigh-abu-dhabi-school/',
    'https://www.edarabia.com/112902/reach-british-school-abu-dhabi-uae/',
    'https://www.edarabia.com/aspen-heights-british-school-ahbs-abu-dhabi-uae/',
    'https://www.edarabia.com/93730/repton-school-abu-dhabi/',
    'https://www.edarabia.com/6391/british-school-al-khubairat/',
    'https://www.edarabia.com/21626/gems-american-academy-abu-dhabi/',
    'https://www.edarabia.com/22370/liwa-international-school-abu-dhabi-uae/',
    'https://www.edarabia.com/spanish-school-abu-dhabi-uae/',
    'htt

# PArt II

In [141]:
last_dicts_city = []
for i in data_links:
    new_dict = {}
    new_dict['City'] = i['City']
    new_dict['Data'] = [d for c in i['Data'] for d in c]
    last_dicts_city.append(new_dict)

In [ ]:
data_faltante

In [185]:
last_dicts_city_faltante = []
for i in data_faltante:
    new_dict = {}
    new_dict['City'] = i['City']
    new_dict['Data'] = [d for c in i['Data'] for d in c]
    last_dicts_city_faltante.append(new_dict)

In [149]:
with open('new_dict.dat','wb') as f:
    pickle.dump(data_links,f)

In [ ]:
# with open('../restaurant/fork.dat','rb') as f:
#     info_r = pickle.load(f)

In [187]:
data_info_f = []

In [190]:
##### data_info = []
# data_r = []

for idx, j in enumerate(last_dicts_city_faltante):
    list_links = list(set(j['Data']))
    print(f'{idx}/{len(last_dicts_city)}')
    num_proxy = 0
    for idx_l,link in enumerate(list_links):
        print(f'{idx_l}/{len(list_links)}')
        if len(lines) == num_proxy:
            print('Se agotaron los proxies')
        for idx_proxy , proxy in enumerate(lines[num_proxy:]):
            idx_proxy += num_proxy
            proxy = proxy.replace('\n','')
            try:
                r = requests.get(link,proxies={'http':proxy,'https':proxy},timeout=3)
                soup = BeautifulSoup(r.text,'lxml')
                school_name = soup.find('h1').text
                try:
                    leadership = soup.body.find(text="Leadership: ").find_parent('li').text.replace('Leadership:','').strip()
                except:
                    leadership = None
                address = remove_blank_spaces(soup.find(text="Address: ").find_parent('li').text.split('\n')[0].replace('(Map)','').replace('Address:',''))
                try:
                    tel = remove_blank_spaces(soup.find(text="Tel: ").find_parent('li').text.replace('Show Number','').replace('Tel:',''))
                except:
                    tel = None
                try:
                    website = soup.find('a',attrs={'title':"Visit Website"}).get('href')
                except:
                    website = None
                data = {
                'school_name':school_name,
                'Leadership':leadership,
                'Address':address,
                'Tel':tel,
                'Website': website,
                'link':link,
                'City':j['City']}
                data_info_f.append(data)
                data_r.append(r)
                print('\tSuccess')
                num_proxy = idx_proxy
                break
        
            except ConnectionError as cx:
                pass
        
            except Exception as ex:
                print(ex)
                print(link)
                pass
        print('Success B')

0/193
0/8
	Success
Success B
1/8
	Success
Success B
2/8
	Success
Success B
3/8
	Success
Success B
4/8
	Success
Success B
5/8
HTTPSConnectionPool(host='www.edarabia.com', port=443): Read timed out. (read timeout=3)
https://www.edarabia.com/10303/ozel-gelisim-ilkogretim-okulu/
	Success
Success B
6/8
	Success
Success B
7/8
	Success
Success B
1/193
0/1
	Success
Success B
2/193
0/6
	Success
Success B
1/6
	Success
Success B
2/6
	Success
Success B
3/6
	Success
Success B
4/6
	Success
Success B
5/6
	Success
Success B
3/193
0/13
	Success
Success B
1/13
	Success
Success B
2/13
HTTPSConnectionPool(host='www.edarabia.com', port=443): Read timed out. (read timeout=3)
https://www.edarabia.com/st-bernard-preparatory-school-cullman-us/
HTTPSConnectionPool(host='www.edarabia.com', port=443): Read timed out. (read timeout=3)
https://www.edarabia.com/st-bernard-preparatory-school-cullman-us/
HTTPSConnectionPool(host='www.edarabia.com', port=443): Read timed out. (read timeout=3)
https://www.edarabia.com/s

In [151]:
data_info

[{'school_name': 'Merryland International School',
  'Leadership': 'Mr. Aaron Grandon (Chairman)',
  'Address': 'Plot 13 ME‐9 Musaffah - Abu Dhabi UAE',
  'Tel': '+97125519626',
  'Website': 'http://www.merrylandinternational.net',
  'link': 'https://www.edarabia.com/76360/merryland-international-school/',
  'City': 'abu-dhabi'},
 {'school_name': 'Cranleigh Abu Dhabi School ',
  'Leadership': 'Mr. Michael Wilson (Principal)',
  'Address': 'St Regis Residences Saadiyat - Abu Dhabi UAE',
  'Tel': '+97124970000',
  'Website': 'http://www.cranleigh.ae',
  'link': 'https://www.edarabia.com/82124/cranleigh-abu-dhabi-school/',
  'City': 'abu-dhabi'},
 {'school_name': 'Reach British School Abu Dhabi ',
  'Leadership': 'Mr. Dene Bright (Principal)',
  'Address': 'Baniyas East - Abu Dhabi UAE',
  'Tel': '+97125822030',
  'Website': 'http://www.reachbritishschool.com',
  'link': 'https://www.edarabia.com/112902/reach-british-school-abu-dhabi-uae/',
  'City': 'abu-dhabi'},
 {'school_name': 'Ajman 

In [191]:
with open('data_faltante.dat','wb') as f:
    pickle.dump(data_info_f,f)

In [153]:
data_copy = data_info.copy()

In [197]:
data_copy_f = data_copy + data_info_f

In [198]:
data_copy_f

[{'school_name': 'Merryland International School',
  'Leadership': 'Mr. Aaron Grandon (Chairman)',
  'Address': 'Plot 13 ME‐9 Musaffah - Abu Dhabi UAE',
  'Tel': '+97125519626',
  'Website': 'http://www.merrylandinternational.net',
  'link': 'https://www.edarabia.com/76360/merryland-international-school/',
  'City': 'abu-dhabi'},
 {'school_name': 'Cranleigh Abu Dhabi School ',
  'Leadership': 'Mr. Michael Wilson (Principal)',
  'Address': 'St Regis Residences Saadiyat - Abu Dhabi UAE',
  'Tel': '+97124970000',
  'Website': 'http://www.cranleigh.ae',
  'link': 'https://www.edarabia.com/82124/cranleigh-abu-dhabi-school/',
  'City': 'abu-dhabi'},
 {'school_name': 'Reach British School Abu Dhabi ',
  'Leadership': 'Mr. Dene Bright (Principal)',
  'Address': 'Baniyas East - Abu Dhabi UAE',
  'Tel': '+97125822030',
  'Website': 'http://www.reachbritishschool.com',
  'link': 'https://www.edarabia.com/112902/reach-british-school-abu-dhabi-uae/',
  'City': 'abu-dhabi'},
 {'school_name': 'Ajman 

In [199]:
for idx, i in enumerate(data_copy_f):
    n = 0
    try:
        data_copy_f[idx] = pd.DataFrame(i)
    except ValueError as e:
        data_copy_f[idx] = pd.DataFrame.from_dict(i,orient='index').transpose()

In [200]:
data_copy_f = pd.concat(data_copy_f)

In [201]:
data_copy_f.drop_duplicates(inplace=True)

In [203]:
data_copy_f.shape

(1184, 7)

In [206]:
df = data_copy_f.drop(columns=['link'])

In [207]:
df.to_excel('adarabia_total.xlsx',index=False)